# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [2]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject

#import packages

import matplotlib as plt
from scipy import optimize
import pandas as pd


In [14]:
model = inauguralproject.inauguralproject()

In [28]:
#create list with new values of alpha and solve
al = [0.25, .5, .75]
#create list with new values of sigma and solve
sl = [0.5, 1, 1.5]
optm = np.zeros([3,3])
for it, a in enumerate(al):
    model.par.alpha = a

    for jt, s in enumerate(sl):
        model.par.sigma = s
        opt = model.solve_discrete()
        optm[it, jt] = opt.HM/opt.HF




c:\Users\Yas\Documents\Uni\Vorlesungen\2023FS\Programming\projects-2023-newteam-1\inauguralproject\inauguralproject.py:61: RuntimeWarning: divide by zero encountered in reciprocal
  
c:\Users\Yas\Documents\Uni\Vorlesungen\2023FS\Programming\projects-2023-newteam-1\inauguralproject\inauguralproject.py:65: RuntimeWarning: invalid value encountered in multiply
  TF = LF+HF
C:\Users\Yas\AppData\Local\Temp\ipykernel_11412\1966323211.py:12: RuntimeWarning: invalid value encountered in double_scalars
  optm[it, jt] = opt.HM/opt.HF


In [29]:
optm

array([[       nan, 3.25      , 8.        ],
       [       nan, 1.        , 0.8       ],
       [       nan, 0.30769231, 0.125     ]])

# Question 1

**Explain how you solve the model**

In [16]:
# code for solving the model (remember documentation and comments)

a = np.array([1,2,3])
b = inauguralproject.square(a)
print(b)

AttributeError: module 'inauguralproject' has no attribute 'square'

# Question 2

Explain your code and procedure

In [3]:
#create list with new values of WF
wFl = [0.8, 0.9, 1.0, 1.1, 1.2]
    for it, w in enumerate(WFl):
    model.par.WF = w

#define Hf/Hm
    x = log () #how to define HF/HM??

#define wf/wm
    y = log (model.par.wF/ model.par.wM)

# plot 
    plt.plot(x,y)

# Set plot labels and title
    plt.xlabel('log of HM/WM')
    plt.ylabel('log of WF/WM')
    plt.title('Logarithmic plot')

# Question 3

Explain your code and procedure

In [4]:
# code

ADD CONCISE CONLUSION.